#### To test: 
1. Create a folder ~/data/luna16/
2. Create a folder ~/data/luna16/hdf5
    -Under this copy UNET_weights_H5.h5 (download from google drive)
3. Create a folder ~/data/luna16/hdf5/HO5 
    -Under this copy 64x64x64-patch-withdiam_sample.hdf5

In [1]:
import numpy as np
import h5py
import pandas as pd
import keras

from ipywidgets import interact

#from UNET_utils import *
from UNET_model_def import *
from matplotlib import pyplot as plt
%matplotlib inline

/Users/sumangunnala/anaconda2/envs/tensorflow_keras/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [33]:
#!pip install ipywidgets

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
pwd

'/Users/sumangunnala/Documents/DSE/sgunnala/Capstone/lung-nodule-localization/src/Inference/patch/inferencecode_old'

In [4]:
HOLDOUT = 5

TOP_DIR = "/Users/sumangunnala/Documents/DSE/sgunnala/Capstone/dataset/luna16_DATA/hdf5files"
HO_dir = '/HO{}/'.format(HOLDOUT)
data_dir = TOP_DIR + HO_dir

#model_wghts = "{}/Test_UNET_weights_modelC_exp1_lr02_BSexp1_H{}.h5".format(data_dir, HOLDOUT)
#model_wghts = "{}/Test_UNET_weights_modelB_exp1_lr02_BSexp1_H{}.h5".format(data_dir, HOLDOUT)
model_wghts = "{}/UNET_weights_modelB_exp1_lr009_BSexp1_H{}.h5".format(data_dir, HOLDOUT)
model = unet3D_Model6_Model13((None,None,None,1))
hu_norm_on = False # set it to True if model has HU normalization at the time of training



3D U-Net Segmentation


In [5]:
model.load_weights(model_wghts)

In [6]:
hdf = h5py.File("{}/64x64x64-patch-withdiam_HO{}sample.hdf5".format(data_dir,HOLDOUT), "r")

In [7]:
list(hdf.keys())

['centroid', 'diameter_label', 'input', 'notrain', 'output', 'subsets', 'uuid']

In [8]:
class1 = np.where(hdf["output"][:,0]==1)[0]  # Where are class 1
class0 = np.where(hdf["output"][:,0]==0)[0]  # Where are class 1

In [9]:
len(class1)

19

In [10]:
def hu_norm(npzarray):
    
    """
    Normalize pixel depth into Hounsfield units (HU), between -1000 - 400 HU
    All other HU will be masked. Then we normalize pixel values between 0 and 1.
    """
    if hu_norm_on:
        maxHU, minHU = 400., -1000.
        npzarray = (npzarray - minHU) / (maxHU - minHU)
        npzarray[npzarray>1] = 1.
        npzarray[npzarray<0] = 0.
    return npzarray

In [11]:
print("Class 1")

class1idx = 8

# Choose one of the class 1 ROIs

img1 = hdf["input"][class1[class1idx],:].reshape(64,64,64,1)  
print(img1.shape)

predict_array = model.predict(hu_norm(np.expand_dims(img1,0)), verbose=2)

prediction_mask1 = np.round(predict_array)


def displaySlice(sliceNo):
    
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.imshow(img1[:,:,sliceNo,0], cmap="bone")
    plt.title("Image")

    plt.subplot(1,3,2)
    plt.imshow(prediction_mask1[0,:,:,sliceNo,0], cmap="bone")
    plt.title("Tumor Prediction")

    plt.subplot(1,3,3)
    plt.imshow(img1[:,:,sliceNo,0], cmap="bone")
    plt.imshow(prediction_mask1[0,:,:,sliceNo,0], alpha=0.5, cmap="Reds")
    plt.title("Overlay")
    
    
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.imshow(img1[sliceNo,:,:,0], cmap="bone")
    plt.title("Image")

    plt.subplot(1,3,2)
    plt.imshow(prediction_mask1[0,sliceNo,:,:,0], cmap="bone")
    plt.title("Tumor Prediction")

    plt.subplot(1,3,3)
    plt.imshow(img1[sliceNo,:,:,0], cmap="bone")
    plt.imshow(prediction_mask1[0,sliceNo,:,:,0], alpha=0.5, cmap="Reds")
    plt.title("Overlay")
    
    
    
interact(displaySlice, sliceNo=(1,img1.shape[2],1)); 

print(hdf["uuid"][0])

Class 1
(64, 64, 64, 1)


A Jupyter Widget

[b'1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800']


In [12]:
print("Class 0")
class0idx = 590

# Choose one of the class 0 ROIs

img0 = hdf["input"][class0[class0idx],:].reshape(64,64,64,1)  


predict_array = model.predict(hu_norm(np.expand_dims(img0,0)), verbose=2)

prediction_mask0 = np.round(predict_array)


def displaySlice(sliceNo):
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.imshow(img0[:,:,sliceNo,0], cmap="bone")
    plt.title("Image")

    plt.subplot(1,3,2)
    plt.imshow(prediction_mask0[0,:,:,sliceNo,0], cmap="bone")
    plt.title("Tumor Prediction")

    plt.subplot(1,3,3)
    plt.imshow(img0[:,:,sliceNo,0], cmap="bone")
    plt.imshow(prediction_mask0[0,:,:,sliceNo,0], alpha=0.5, cmap="Reds")
    plt.title("Overlay")
    
    
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.imshow(img0[sliceNo,:,:,0], cmap="bone")
    plt.title("Image")

    plt.subplot(1,3,2)
    plt.imshow(prediction_mask0[0,sliceNo,:,:,0], cmap="bone")
    plt.title("Tumor Prediction")

    plt.subplot(1,3,3)
    plt.imshow(img0[sliceNo,:,:,0], cmap="bone")
    plt.imshow(prediction_mask0[0,sliceNo,:,:,0], alpha=0.5, cmap="Reds")
    plt.title("Overlay")
    
    
    
interact(displaySlice, sliceNo=(1,img0.shape[2],1)); 

print(hdf["uuid"][0])

Class 0


A Jupyter Widget

[b'1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800']


In [11]:
#hdf.close()

In [12]:
img1.shape

(64, 64, 64, 1)

In [13]:
img0.shape

(64, 64, 64, 1)

In [14]:
prediction_mask0.shape

(1, 64, 64, 64, 1)

In [15]:
prediction_mask1.shape

(1, 64, 64, 64, 1)